In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, rfft, rfftfreq

In [ ]:
grades = pd.read_csv('/content/drive/MyDrive/eeg_grades.csv')

In [ ]:
#@title importing data
# import necessary libraries
import pandas as pd
import os
import glob
  
  
# use glob to get all the csv files 
# in the folder
path = '/content/drive/MyDrive/CSV_format.zip (Unzipped Files)/CSV_format'
csv_files = glob.glob(os.path.join(path, "*.csv.xz"))# this puts the file names in a list
  
dfs = []

# loop over the list of csv files
for f in csv_files:
    name = f.split('format/')[1].split('.')[0]#get name
    k=0
    for id in grades['file_ID']:#for each epoch, find the grade id that matches
      if name ==id:
        index = k
      k+=1
# so we will have the index of the grade in grades for the epoch


    if ((grades['grade']>0)[index]==True)==True: # ie if its not a NaN
      dfs.append([ pd.read_csv(f), grades['grade'][index], grades['file_ID'][index] ]) #append our list with the data, the grade, and the file id

In [ ]:
import pandas as pd
import os
import glob
  
  
# use glob to get all the csv files 
# in the folder
path = '/content/drive/MyDrive/CSV_format.zip (Unzipped Files)/CSV_format'
csv_files = glob.glob(os.path.join(path, "*.csv.xz"))# this puts the file names in a list#


In [ ]:
training_grades=grades.dropna()

In [ ]:
train_windows=[]
train_labels=[]
ds=10 # downsampling rate 
for k in range(0,200):
  t = np.random.randint(1,90) # pick a random epoch
  df=pd.read_csv(path +'/'+np.array(training_grades['file_ID'])[t]+'.csv.xz')
  ds=len(df)/15000
  df = df.iloc[[ds*i for i in range(0,int(np.floor(len(df)/ds)))]] #import and downsample by 60 times

  n = np.random.randint(10)

  wind = df.iloc[0 +int((len(df)/12)*(1/2)*n):int((len(df)/12))+int((len(df)/12)*(1/2)*n)] # find a random window of length 5 minutes

  train_windows.append(np.array(wind))
  train_labels.append(np.array(training_grades['grade'])[t])

In [ ]:
train_labels=np.array(train_labels, dtype=int)-1

In [ ]:
train_windows=np.array(train_windows, dtype=np.float64)

In [ ]:
hot_train_labels=[]
for i in range(0,len(train_labels)):
  hot_y=[]
  for j in range(0,train_labels[i]):
    hot_y.append(0)
  hot_y.append(1)
  for j in range(train_labels[i]+1, 4):
    hot_y.append(0)
  hot_train_labels.append(hot_y)

In [ ]:
for i in range(0,10):
  train_windows[:,:,i]=(train_windows[:,:,i]-np.min(train_windows[::,i]))/(np.max(train_windows[::,i])-np.min(train_windows[::,i]))

In [ ]:
model = Sequential([
    TCN(input_shape=(1250,9), nb_filters=256, return_sequences=True, dilations =[1,2,4,8,16]),
    Flatten(),
    Dense(32,activation='relu'),
    Dense(4, activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 1250, 256)         1781504   
                                                                 
 flatten (Flatten)           (None, 320000)            0         
                                                                 
 dense (Dense)               (None, 32)                10240032  
                                                                 
 dense_1 (Dense)             (None, 4)                 132       
                                                                 
Total params: 12,021,668
Trainable params: 12,021,668
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = 'accuracy')

In [ ]:
model.fit(train_windows[:,:,1:],np.array(hot_train_labels), epochs = 100, batch_size=8)

Epoch 1/100
25/25 [==============================] - 1s 51ms/step - loss: 0.9781 - accuracy: 0.5950
Epoch 2/100
25/25 [==============================] - 1s 48ms/step - loss: 0.8724 - accuracy: 0.6250
Epoch 3/100
25/25 [==============================] - 1s 48ms/step - loss: 3.1453 - accuracy: 0.5550
Epoch 4/100
25/25 [==============================] - 1s 48ms/step - loss: 2.4702 - accuracy: 0.4850
Epoch 5/100
25/25 [==============================] - 1s 48ms/step - loss: 1.2157 - accuracy: 0.5400
Epoch 6/100
25/25 [==============================] - 1s 49ms/step - loss: 1.1424 - accuracy: 0.5400
Epoch 7/100
25/25 [==============================] - 1s 48ms/step - loss: 1.0910 - accuracy: 0.6050
Epoch 8/100
25/25 [==============================] - 1s 48ms/step - loss: 1.2342 - accuracy: 0.5900
Epoch 9/100
25/25 [==============================] - 1s 48ms/step - loss: 0.9202 - accuracy: 0.6350
Epoch 10/100
25/25 [==============================] - 1s 48ms/step - loss: 0.9264 - accuracy: 0.6500

In [ ]:
preds=model.predict(train_windows[:,:,1:])

In [ ]:
preds_class =[]
for i in range(0,len(preds)):
  preds_class.append(np.argmax(preds[i]))
preds_class=np.array(preds_class)
preds_class=preds_class.reshape(len(preds_class))

In [ ]:
preds_class

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

In [ ]:
train_labels

array([2, 3, 2, 0, 0, 3, 0, 0, 0, 3, 3, 1, 0, 0, 0, 0, 1, 0, 2, 0, 2, 2,
       0, 3, 1, 1, 0, 0, 3, 0, 0, 1, 1, 0, 0, 3, 0, 3, 0, 0, 0, 1, 0, 0,
       2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0,
       3, 1, 3, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 2, 2, 0, 0, 0,
       0, 1, 0, 0, 2, 0, 0, 1, 1, 0, 0, 0, 0, 3, 0, 2, 2, 2, 0, 1, 0, 3,
       0, 0, 0, 3, 1, 0, 2, 0, 0, 0, 0, 1, 0, 3, 3, 1, 3, 1, 1, 0, 1, 1,
       3, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 0, 2, 0,
       2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 1, 0, 2, 1, 0, 3,
       2, 0])

In [ ]:
np.sum((preds_class==train_labels)*1)/len(preds)

0.605

#Model Creation

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D, AveragePooling2D, Dropout,GlobalAveragePooling2D, MaxPooling1D 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [ ]:
def create_convnet():
    # layers as specified in the paper
    input_shape = tf.keras.Input(shape=(256, 112, 1))

    tower_1 = Conv2D(10, (8, 1), padding='same', activation='relu')(input_shape)
    tower_1 = MaxPooling2D((4, 4), strides=(2, 2), padding='same')(tower_1)

    tower_2 = Conv2D(10, (1, 8), padding='same', activation='relu')(input_shape)
    tower_2 = MaxPooling2D((4, 4), strides=(2, 2), padding='same')(tower_2)

    tower_3 = Conv2D(10, (8, 8), padding='same', activation='relu')(input_shape)
    tower_3 = MaxPooling2D((4, 4), strides=(2, 2), padding='same')(tower_3)

    merged = keras.layers.concatenate([tower_1, tower_2, tower_3], axis=3)

    layer1 = Conv2D(60, (4,4), padding='same', 
                    activation ='relu',strides =(2,2))(merged)
    
    layer2a = MaxPooling2D((2,2), padding ='same', strides =(2,2))(layer1)
    layer2b = BatchNormalization()(layer2a)
    layer3 = Conv2D(60, (2,2), padding='same' )(layer2b)

    layer4 = MaxPooling2D( (2,2), padding='same', strides =(2,2))(layer3)
    layer5 = GlobalAveragePooling2D()(layer4)

    out = Dense(60, activation='relu')(layer5)
    out = Dense(4, activation='softmax')(out)

    model = tf.keras.Model(input_shape, out)
    #plot_model(model, to_file=img_path)
    return model

In [ ]:
model =create_convnet()

In [ ]:
model.summary()

In [ ]:
#defining the learning rate step
def scheduler(epoch, lr):
  n =np.floor((epoch-1)/5)
  return lr*(0.8)**n
opt = tf.keras.optimizers.SGD(momentum =0.9, nesterov =True)
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

In [ ]:
model.compile(opt,loss='categorical_crossentropy')

In [ ]:
#model.fit(x=x, y=y, epochs=30, batch_size=128, callbacks=[callback])

In [ ]:
def mpredict(x):
  return model.predict(x)

In [ ]:
x = np.random.rand(256,112,6)

In [ ]:
n_channels = 6

In [ ]:
#voting mechanism, takes in one QTFD image across 6 channels and ouputs the guess from each channel
def ImageVote(x):
  channel_preds = []
  for channel_i in range(0, n_channels): # for each channel
    channel_preds.append( np.argmax(mpredict(np.array([ x[:,:,channel_i] ]) )))
  return channel_preds 

In [ ]:
ImageVote(x)


[3, 3, 3, 3, 3, 3]

# TCN Model

In [ ]:
!pip install -q keras-tcn

     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
from tcn import TCN, tcn_full_summary

In [ ]:
from keras.datasets import mnist

In [ ]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
train_x[0].shape

(28, 28)

In [ ]:
model = Sequential([
    TCN(input_shape=(28,28), nb_filters=256, return_sequences=True, dilations =[1,2,4]),
    Flatten(),
    Dense(10, activation='softmax')
])

In [ ]:
all_hot=[]
for i in range(0,len(train_y)):
  hot_y=[]
  for j in range(0,train_y[i]):
    hot_y.append(0)
  hot_y.append(1)
  for j in range(train_y[i]+1, 10):
    hot_y.append(0)
  all_hot.append(hot_y)

In [ ]:
all_hot_pred=[]
for i in range(0,len(test_y)):
  hot_y=[]
  for j in range(0,test_y[i]):
    hot_y.append(0)
  hot_y.append(1)
  for j in range(test_y[i]+1, 10):
    hot_y.append(0)
  all_hot_pred.append(hot_y)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss = 'categorical_crossentropy')

In [ ]:
model.fit(train_x,np.array(all_hot), epochs = 10, batch_size=256)

In [ ]:
preds=model.predict(test_x)

In [ ]:
preds.shape

(10000, 10)

In [ ]:
preds_class =[]
for i in range(0,len(preds)):
  preds_class.append(np.argmax(preds[i]))

In [ ]:
np.sum((preds_class==test_y)*1)/len(preds)

0.9712